##  Exploring the significance of various factors in determining presence of Heart Disease
Title: Exploring the relative significance of various factors in determining presence of Heart Disease.
## Introduction
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
High blood pressure, unhealthy cholesterol levels, smoking status, and resting heart rate are some of the main predictors of heart disease (CDC). Blood pressure, when too high, can put stress on the arteries in addition to organs such as the heart and kidneys. Cholesterol is a fat-like substance that can build up on the walls of the arteries and reduce blood flow to the heart. Smoking can raise blood pressure and damage the heart and blood vessels. These variables can be changed via medications or lifestyle changes and are therefore worth studying as predictors of heart disease.
## Clearly state the question you will try to answer with your project
Which of the well known predictors of heart disease have the greatest impact on presence? 
Identify and describe the dataset that will be used to answer the question
We will be using the Heart Disease Dataset which contains variables of sex, age, cholesterol, blood pressure, and smoking status.
## Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Read from web!!!
Clean and wrangle your data into a tidy format
Remove all columns except cholesterol (chol), age, sex, blood pressure (trestbps), smoker (smoke), and diagnosis of heart disease (num). 
choose age in which to remove everything under.
Change num into a factor if needed.
## Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Use above data ^, group by num, summarize mean smth
## Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.
Plot age vs heart disease, decide to cut off at some age.
Methods:


## Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
We will use cholesterol, resting heart rate, blood pressure, smoker, sex.
## Describe at least one way that you will visualize the results
We will separate into male and female datasets and use k-nearest-neighbours to predict heartdisease and see which are the most ?correct? With testing datatset. From there we can see which impact it the most. So we can plot each variable we are keeping against heart disease, with different graphs for sex?
## Expected outcomes and significance:


## What do you expect to find? 
We expect to find that high cholesterol and high blood pressure are linked to positive diagnoses of heart disease. 
## What impact could such findings have?
knowledge on what you can change about your lifestyle to prevent risk of heart disease. 
## What future questions could this lead to?
## What factors about our diet and lifestyles can be changed to reduce these high-levels. What factors are most important to be tested early and often in order to change them before onset of disease.
 Which factors are most prominent in positive diagnoses

In [1]:
library(tidyverse)
library(tidymodels)
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
file_path <- "processed.cleveland.data"
column_names <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "target")
download.file(url, destfile = file_path, method = "auto")
dataset <- read_delim("processed.cleveland.data", delim = ",", col_names = column_names)
dataset


<<<<<<< local
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0
62,0,4,140,268,0,2,160,0,3.6,3,2.0,3.0,3
57,0,4,120,354,0,0,163,1,0.6,1,0.0,3.0,0
63,1,4,130,254,0,2,147,0,1.4,2,1.0,7.0,2


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt